In [9]:
import ee
import numpy as np
import urllib.request
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


In [10]:
ee.Authenticate()
ee.Initialize(project='ee-tiagobettervide')

In [11]:
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
AOI = countries.filter(ee.Filter.eq('country_na', 'Indonesia'))


wrs2_descending = ee.FeatureCollection('projects/ee-elf-cloud/assets/WRS2_descending_0')


wrs2_AOI = wrs2_descending.filterBounds(AOI)


def addNDVI(image):
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)


imgColT1 = ee.ImageCollection('LANDSAT/LT05/C02/T1').filterBounds(AOI).map(addNDVI)
imgColT2 = ee.ImageCollection('LANDSAT/LT05/C02/T2').filterBounds(AOI).map(addNDVI)


ndvi_palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301'
]


pdf_filename = 'Indonesia_NDVI_1985_1998_PATH_ROW.pdf'


south_america_aoi = AOI.geometry().bounds().getInfo()['coordinates']

empty = ee.Image().byte()
outline = empty.paint(featureCollection=wrs2_AOI, color=1, width=2)

def add_outline_to_mosaic(mosaic):
    return mosaic.blend(outline)

# Criar o PDF
with PdfPages(pdf_filename) as pdf:
    for year in range(1985, 1998):
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'

        imgColT1_year = imgColT1.filterDate(start_date, end_date)
        imgColT2_year = imgColT2.filterDate(start_date, end_date)


        if imgColT1_year.size().getInfo() > 0 and imgColT2_year.size().getInfo() > 0:
            mosaicoT1 = imgColT1_year.qualityMosaic('NDVI').select('NDVI')
            mosaicoT2 = imgColT2_year.qualityMosaic('NDVI').select('NDVI')

            mosaicoT1_com_outline = add_outline_to_mosaic(mosaicoT1)
            mosaicoT2_com_outline = add_outline_to_mosaic(mosaicoT2)


            fig, axs = plt.subplots(1, 2, figsize=(18, 6))


            for mosaic, ax, title in [(mosaicoT1_com_outline, axs[0], f'NDVI T1 - {year}'),
                                       (mosaicoT2_com_outline, axs[1], f'NDVI T2 - {year}')]:


                img_thumbnail = mosaic.getThumbUrl(params={
                    'min': 0,
                    'max': 1,
                    'palette': ndvi_palette,
                    'region': south_america_aoi,
                    'dimensions': '1000',
                    'format': 'png'
                })


                with urllib.request.urlopen(img_thumbnail) as response:
                    img = Image.open(response)
                    img_array = np.array(img)


                ax.imshow(img_array, aspect='auto')
                ax.set_title(title, fontsize=16)
                ax.axis('off')


            for feature in wrs2_AOI.getInfo()['features']:
                properties = feature['properties']
                wrspr = properties.get('WRSPR', 'N/A')


                geometry = ee.Geometry(feature['geometry'])
                centroid = geometry.centroid().getInfo()['coordinates']


                x_coord = centroid[0]
                y_coord = centroid[1]


                bounds = AOI.geometry().bounds().getInfo()['coordinates'][0]
                min_x = min(coord[0] for coord in bounds)
                max_x = max(coord[0] for coord in bounds)
                min_y = min(coord[1] for coord in bounds)
                max_y = max(coord[1] for coord in bounds)

                img_width = img_array.shape[1]
                img_height = img_array.shape[0]

                x_img = int((x_coord - min_x) / (max_x - min_x) * img_width)
                y_img = int(img_height - (y_coord - min_y) / (max_y - min_y) * img_height)


                axs[0].text(x_img, y_img, f'{wrspr}', fontsize=7, color='red', ha='center', va='center',
                            bbox=dict(facecolor='white', alpha=0), rotation=15)

                axs[1].text(x_img, y_img, f'{wrspr}', fontsize=7, color='red', ha='center', va='center',
                            bbox=dict(facecolor='white', alpha=0), rotation=15)


            pdf.savefig(fig)
            plt.close()
        else:
            print(f'Nenhuma imagem disponível para o ano {year}.')

print(f'PDF criado com sucesso: {pdf_filename}')


Nenhuma imagem disponível para o ano 1985.
PDF criado com sucesso: Indonesia_NDVI_1985_1998_PATH_ROW.pdf


In [ ]:
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
AOI = countries.filter(ee.Filter.eq('country_na', 'Indonesia'))


wrs2_descending = ee.FeatureCollection('projects/ee-elf-cloud/assets/WRS2_descending_0')

south_america_aoi = AOI.geometry().bounds().getInfo()['coordinates']

def addNDVI(image):
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)


imgColT1 = ee.ImageCollection('LANDSAT/LT05/C02/T1').filterBounds(AOI).map(addNDVI)
imgColT2 = ee.ImageCollection('LANDSAT/LT05/C02/T2').filterBounds(AOI).map(addNDVI)


ndvi_palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301'
]


pdf_filename = 'Indonesia_NDVI_1985_1998.pdf'


with PdfPages(pdf_filename) as pdf:
    for year in range(1985, 1998):
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'


        imgColT1_year = imgColT1.filterDate(start_date, end_date)
        imgColT2_year = imgColT2.filterDate(start_date, end_date)


        if imgColT1_year.size().getInfo() > 0 and imgColT2_year.size().getInfo() > 0:

            mosaicoT1 = imgColT1_year.qualityMosaic('NDVI').select('NDVI')
            mosaicoT2 = imgColT2_year.qualityMosaic('NDVI').select('NDVI')


            fig, axs = plt.subplots(1, 2, figsize=(18, 6))


            for mosaic, ax, title in [(mosaicoT1, axs[0], f'NDVI T1 - {year}'),
                                       (mosaicoT2, axs[1], f'NDVI T2 - {year}')]:


                img_thumbnail = mosaic.getThumbUrl(params={
                    'min': 0,
                    'max': 1,
                    'palette': ndvi_palette,
                    'region': south_america_aoi,
                    'dimensions': '1000',
                    'format': 'png'
                })


                with urllib.request.urlopen(img_thumbnail) as response:
                    img = Image.open(response)
                    img_array = np.array(img)


                ax.imshow(img_array, aspect='auto')
                ax.set_title(title, fontsize=16)
                ax.axis('off')




            pdf.savefig(fig)
            plt.close()
        else:
            print(f'Nenhuma imagem disponível para o ano {year}.')

print(f'PDF criado com sucesso: {pdf_filename}')


Nenhuma imagem disponível para o ano 1985.
PDF criado com sucesso: Indonesia_NDVI_1985_1998
